Data Preprocessing

In [2]:
import numpy as np
import pandas as pd 
import tensorflow as tf

dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[ : , 3:-1].values
y = dataset.iloc[ : , -1].values

In [3]:
# Label encoding the 'Gender' column

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[ :, 2] = le.fit_transform(X[ :, 2])

In [4]:
# One-hot encoding the 'Geography' column

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X)) 

In [5]:
print(len(X[0]))

12


In [6]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [7]:
# Splitting dataset into training and testing set

from sklearn.model_selection import train_test_split    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [8]:
print(len(X_train[0]))

12


In [9]:
# Feature scaling (very important/compulsory for Deep-Learning)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

---

Building the ANN

In [10]:
ann = tf.keras.models.Sequential()

# adding the input layer and first hidden layer (input layer is added automatically) 
# we choose the no. of neurons for hidden layer as 6, based on trial & error; relu=rectifier activation fn.
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

# adding a second hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

# adding the output layer, 1 neuron in O/P layer; activation fn. as sigmoid since we want the probability of a person exiting
# if we had 3 outcomes, A,B,C then we would need 3 neurons- A(1,0,0); B(0,1,0); C(0,0,1)

# binary classification: sigmoid; non-binary: softmax
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

---

Training the ANN

In [11]:
# compiling the ANN
# binary classification: binary_crossentropy; non-binary: categorical_crossentropy
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5336 - accuracy: 0.7741
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4627 - accuracy: 0.7996
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4351 - accuracy: 0.8102
Epoch 4/100
250/250 [==============================] - 0s 800us/step - loss: 0.4173 - accuracy: 0.8206
Epoch 5/100
250/250 [==============================] - 0s 776us/step - loss: 0.4016 - accuracy: 0.8322
Epoch 6/100
250/250 [==============================] - 0s 771us/step - loss: 0.3880 - accuracy: 0.8369
Epoch 7/100
250/250 [==============================] - 0s 766us/step - loss: 0.3782 - accuracy: 0.8438
Epoch 8/100
250/250 [==============================] - 0s 775us/step - loss: 0.3714 - accuracy: 0.8474
Epoch 9/100
250/250 [==============================] - 0s 762us/step - loss: 0.3665 - accuracy: 0.8505
Epoch 10/100
250/250 [==============================] - 0s 901us/step - loss: 0

Epoch 79/100
250/250 [==============================] - 0s 819us/step - loss: 0.3342 - accuracy: 0.8622
Epoch 80/100
250/250 [==============================] - 0s 902us/step - loss: 0.3336 - accuracy: 0.8622
Epoch 81/100
250/250 [==============================] - 0s 885us/step - loss: 0.3338 - accuracy: 0.8627
Epoch 82/100
250/250 [==============================] - 0s 884us/step - loss: 0.3345 - accuracy: 0.8616
Epoch 83/100
250/250 [==============================] - 0s 906us/step - loss: 0.3340 - accuracy: 0.8596
Epoch 84/100
250/250 [==============================] - 0s 858us/step - loss: 0.3336 - accuracy: 0.8622
Epoch 85/100
250/250 [==============================] - 0s 878us/step - loss: 0.3338 - accuracy: 0.8619
Epoch 86/100
250/250 [==============================] - 0s 846us/step - loss: 0.3333 - accuracy: 0.8602
Epoch 87/100
250/250 [==============================] - 0s 803us/step - loss: 0.3340 - accuracy: 0.8608
Epoch 88/100
250/250 [==============================] - 0s 794us

---

Predicting a single observation

In [12]:
# Dummy data: France, 600, Male, 40, 3, 60000, 2, 1, 1, 50000
# we set threshold as 0.5; >0.5=1; <0.5=0
print(ann.predict(sc.transform([[1.0, 0.0, 0.0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


---

Predicting the Test set result and evaluating the model

In [13]:
y_pred = ann.predict(X_test) > 0.5
print(y_pred)

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


In [14]:
print(np.concatenate((y_pred.reshape(-1,1), y_test.reshape(-1,1)),1))

[[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


---

Making the Confusion Matrix

In [15]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1527   58]
 [ 213  202]]


0.8645